# LSIによる次元圧縮の動作確認

ストーリー: https://www.pivotaltracker.com/story/show/149341617

LSI_in_sckit_learn_3.ipynbの続きで、ボキャブラリーのall.csvを追加することでどのように結果が変わるか実験

In [1]:
import sys
import os

os.getcwd()

'/Users/shwld/project/mofmof/donusagi-bot/learning/prototype'

In [2]:
learning_dir = os.path.abspath("../")
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [3]:
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# [追加処理] ボキャブラリーの用意

In [4]:
all_df = pd.read_csv('prototype/fixtures/learning_training_messages/all.csv')
all_sentences = np.array(all_df['question'])
all_separated_sentences = Nlang.batch_split(all_sentences)
all_separated_sentences
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
all_features = vectorizer.fit_transform(all_separated_sentences)

## データの準備

In [5]:
df = pd.read_csv('./prototype/fixtures/learning_training_messages/LSI_in_sckit_learn.csv')
sentences = np.array(df['question'])
separated_sentences = Nlang.batch_split(sentences)
separated_sentences

['育児 短時間 勤務 申請 書 いつ 提出 する',
 '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
 '育児 短時間 勤務 申請 どこ 提出 する よい',
 '子供 生まれる',
 'ＶＩＳＡ 勘定 科目 わかる ない',
 '海外 出張 する 旅費 精算 する どう する 良い',
 '保険証 失 くい']

## TfidfVectorizerを用いてベクトルを作る

In [6]:
features = vectorizer.transform(separated_sentences)

## 検索する単語も同様に処理する

In [7]:
target_sentences = np.array(['こどもがうまれた'])
target_separated_sentences = Nlang.batch_split(target_sentences)
target_features = vectorizer.transform(target_separated_sentences)
print(target_separated_sentences)
target_sentences2 = np.array(['うまれたばかりの赤ちゃんを抱いた'])
target_separated_sentences2 = Nlang.batch_split(target_sentences2)
target_features2 = vectorizer.transform(target_separated_sentences2)
target_separated_sentences2

['こども うまれる']


['うまれる 赤ちゃん 抱く']

## コサイン類似検索をかけてみる(LSIなし)

In [8]:
similarities = cosine_similarity(features, target_features)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]

In [9]:
similarities = cosine_similarity(features, target_features2)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]

## TruncatedSVDで次元圧縮する

In [10]:
lsa = TruncatedSVD(n_components=6, algorithm='randomized', n_iter=10, random_state=42)
lsa.fit(all_features)
lsa_features = lsa.transform(features)
normalizer = Normalizer(copy=False)
lsa_features = normalizer.fit_transform(lsa_features)

In [11]:
lsa_target_features = lsa.transform(target_features)
lsa_target_features = normalizer.transform(lsa_target_features)
lsa_target_features2 = lsa.transform(target_features2)
lsa_target_features2 = normalizer.transform(lsa_target_features2)

In [12]:
# LSI無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LSIあり
print(lsa_features.shape)
print(lsa_target_features.shape)
print(lsa_target_features2.shape)

(7, 2945)
(1, 2945)
(1, 2945)
(7, 6)
(1, 6)
(1, 6)


## コサイン類似検索をかけてみる(LSIあり)

In [13]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]

In [14]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features2)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]

# LDAも試してみる

試せそうだったので `LDA_in_scikit_learn_xx.ipynb` あたりを参考にLDAを試して比較してみた。

http://hivecolor.com/id/65

によると、

> LSIもLDAも、次元をトピック単位に縮約するのは同じでした。lsiコーパスとldaコーパスの値もほとんど同じです。ただ、全く同じではなく、LSIとLDAには、「確率的な揺らぎを考慮するかどうか」という違いがあります。

> LSIでは、一緒に使われている単語であれば、そのうち片方の単語で検索すると、もう片方の単語の結果も見付かることをちょっと前に書きました。しかしながら、どれだけ意味が似ている単語であっても、一緒に使われていなければそういう検索はできないという弱点があります。

> LDAでは、LSIに確率分布を取り入れることで、直接的には一緒に使われていない単語同士であっても検索で見付けることができるようになります。

> これまた詳しくないので詳細な説明はできないのですが、利用者の立場としては、LSIの発展版がLDAだ、くらいに思っておけばいいんじゃないかと思います。

のように書いてあります。

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=8, 
                                max_iter=5,
                                learning_method='online')

lda.fit(all_features)
lda_features = lda.transform(features)
lda_features = normalizer.fit_transform(lda_features)

In [16]:
lda_target_features = lda.transform(target_features)
lda_target_features = normalizer.transform(lda_target_features)
lda_target_features2 = lda.transform(target_features2)
lda_target_features2 = normalizer.transform(lda_target_features2)

In [17]:
# LDA無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LDAあり
print(lda_features.shape)
print(lda_target_features.shape)
print(lda_target_features2.shape)

(7, 2945)
(1, 2945)
(1, 2945)
(7, 8)
(1, 8)
(1, 8)


## コサイン類似検索をかけてみる(LDAあり)

In [18]:
lda_similarities = cosine_similarity(lda_features, lda_target_features)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.70141489349102015, '保険証 失 くい', '保険証を失くした'),
 (0.65582999615287074,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.64529013998095408, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.64441446184776363, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.57396131054704169, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.54255708967841776, '子供 生まれる', '子供が生まれた'),
 (0.45921902111447244, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]

## コサイン類似検索をかけてみる(LDAあり)

In [19]:
lda_similarities = cosine_similarity(lda_features, lda_target_features2)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.70141489349102015, '保険証 失 くい', '保険証を失くした'),
 (0.65582999615287074,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.64529013998095408, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.64441446184776363, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.57396131054704169, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.54255708967841776, '子供 生まれる', '子供が生まれた'),
 (0.45921902111447244, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]

パラメータ調整
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
を参考にパラメータを調整してみた。

何か間違っているのかもしれないが、色々調整してみても、上記のように全く予測できていない。

# ボキャブラリーの影響調査についての結果

- ボキャブラリーが増え、余計な単語が増えてしまったことにより既存のままだと精度が低くなってしまった?
- LSIを使っている場合は、次元削減により一定に削減されるため精度が低くなることはなかった。
- ボキャブラリーが増えたことによりLSIで関連性の有りそうな質問が見つかるようになった。 <- 重要

# LSIについての調査結果


TF-IDFによって単語の重み(出現頻度)がわかるが、SVDで次元圧縮することにより、Questionの他の単語とどのくらい関連があるかを見て近い意味の単語を近いベクトルとして計算できる。
マイオペの場合、検索語も検索対象も単語数が少ないことが多いため、関連がある(一緒に使われている)かどうかでの検索での改善が若干難しそう。
-> 「こんにちは」、「こんにちわ」が一致してくれないのはどちらも1単語だからだと思われる。

質問するときは少なくとも、1単語プラス動詞とかでやったほうが改善する。

LSIを利用しないままだと、おそらく質問が増えるほどにsimilalityが落ちていくと思われるので、LSIを入れておいたほうが良さそう。

一般的な単語と、それを使った質問例文がボキャブラリーとしてfitできると、ひらがな・カタカナの違いなどは改善しそう。



調査時に参考にしたページ

https://nlp.stanford.edu/IR-book/html/htmledition/latent-semantic-indexing-1.html
http://elsur.jpn.org/reading_notes/Greenacre1984.pdf
http://qiita.com/yuku_t/items/483b56be83a3a5423b09
https://ja.wikipedia.org/wiki/%E7%89%B9%E7%95%B0%E5%80%A4%E5%88%86%E8%A7%A3